In [96]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

## Escolha das ferramentas:
https://python.langchain.com/v0.2/docs/integrations/tools/

### Wikipedia para que o agente seja capaz de realizar pesquisas

In [97]:
!pip install wikipedia

In [98]:
from langchain_core.tools import Tool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=3000))
print(wikipedia.run("Paulo Henrique Ganso"))

Page: Paulo Henrique Ganso
Summary: Paulo Henrique Chagas de Lima (born 12 October 1989), known as Paulo Henrique Ganso or just Ganso (lit. "goose" in Portuguese), is a Brazilian professional footballer who plays for Fluminense as an attacking midfielder.
A Santos youth graduate, Ganso was a part of the club's 2010 squad which earned many plaudits due to its playing style, and lifted a number of trophies with the club which included the 2011 Copa Libertadores. In 2012, he moved to São Paulo, being named the best midfielder in the 2014 Campeonato Brasileiro Série A before joining Sevilla in 2016. He could not establish himself as a starter at the Spanish side, and in 2019, after a brief period on loan at Amiens, he returned to his home country with Fluminense.
In 2009, Ganso received a nomination for the Brazilian Football Confederation award for "Breakthrough Player" of the Campeonato Brasileiro Série A, given to the best rookie of the competition.


In [99]:
print(wikipedia.run("Vlad 3"))

Page: Vlad the Impaler
Summary: Vlad III, commonly known as Vlad the Impaler (Romanian: Vlad Țepeș [ˈvlad ˈtsepeʃ]) or Vlad Dracula (; Romanian: Vlad Drăculea [ˈdrəkule̯a]; 1428/31 – 1476/77), was Voivode of Wallachia three times between 1448 and his death in 1476/77. He is often considered one of the most important rulers in Wallachian history and a national hero of Romania.
He was the second son of Vlad Dracul, who became the ruler of Wallachia in 1436. Vlad and his younger brother, Radu, were held as hostages in the Ottoman Empire in 1442 to secure their father's loyalty. Vlad's eldest brother Mircea and their father were murdered after John Hunyadi, regent-governor of Hungary, invaded Wallachia in 1447. Hunyadi installed Vlad's second cousin, Vladislav II, as the new voivode. Hunyadi launched a military campaign against the Ottomans in the autumn of 1448, and Vladislav accompanied him. Vlad broke into Wallachia with Ottoman support in October, but Vladislav returned, and Vlad sough

### Definição da Tool
- A descrição é a parte mais importante do agente

In [100]:
wikipedia_tool = Tool(name = "wikipedia",
                      description="You must never search for multiple concepts at a single step, you need to search one at a time. When asked to compare two concepts for example, search for each one individually.",
                      func=wikipedia.run)

### Criando Tool customizada
- Para um tipo de problema específico
- Faremos uma simples para retornar o dia atual

In [101]:
def current_day(*args, **kwargs):
  from datetime import date
  dia = date.today()
  dia = dia.strftime('%d/%m/%Y')
  return dia

date_tool = Tool(name = 'Day', func = current_day, description = 'Use when need to know the current date')

In [102]:
tools = [wikipedia_tool, date_tool]

# ReAct (Reasoning and Actions)
É uma abordagem proposta para o desenvolvimento de agentes de IA que utilizam modelos de linguagem. A ideia central do ReAct é combinar, de forma integrada, o raciocínio passo a passo (chain-of-thought) com a capacidade de executar ações ou consultas em tempo real, conforme o agente “pensa” em uma solução.
Em termos práticos:

- O agente produz um raciocínio estruturado ou “rastro de raciocínio” (reasoning trace), explicitando seu passo a passo para resolver um problema.
- Ao longo desse raciocínio, o agente pode executar ações (actions), como consultar uma base de dados, fazer buscas, chamar APIs ou realizar cálculos auxiliares, a fim de obter mais informações.
- Por fim, o modelo combina essas informações adicionais com seu raciocínio contínuo, resultando em uma resposta mais sólida e com base em evidências.

## Carregando LLM GPT 4-o Mini

In [104]:
!pip install -q langchain-openai

In [105]:
from langchain_core.tools import Tool
from langchain.tools import WikipediaQueryRun, BaseTool
from langchain_community.utilities import WikipediaAPIWrapper
import getpass
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.agents import AgentExecutor, create_react_agent

https://platform.openai.com/settings/organization/api-keys

OPENAI_API_KEY

In [106]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI Key: ")

OpenAI Key: ··········


In [107]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

agent = create_react_agent(llm = llm, tools = tools, prompt = prompt, stop_sequence=True)
agente_executor = AgentExecutor.from_agent_and_tools(agent = agent, tools = tools, verbose = True, handling_parsing_errors = True)

In [108]:
resp = agente_executor.invoke({"input": "Qual a população de Shenzen?"})



> Entering new AgentExecutor chain...
Para responder à pergunta sobre a população de Shenzhen, vou procurar informações atualizadas sobre a cidade. 

Action: wikipedia  
Action Input: 'Shenzhen'  Page: Shenzhen
Summary: Shenzhen is a city in the province of Guangdong, China. A special economic zone, it is located on the east bank of the Pearl River estuary on the central coast of Guangdong, bordering Hong Kong to the south, Dongguan to the north, Huizhou to the northeast, and Macau to the southwest. With a population of 17.5 million in 2020, Shenzhen is the third most populous city by urban population in China after Shanghai and Beijing. The Port of Shenzhen is the world's fourth busiest container port.
Shenzhen roughly follows the administrative boundaries of Bao'an County, which was established in imperial times. After the Opium Wars, the southern portion of Bao'an County was occupied by the British government and became part of British Hong Kong, while the village of Shenzhen was 

## Tavily

TAVILY_API_KEY: https://app.tavily.com/home?code=2XezeE8-VqOWx-oPtvXqYMGZaIJZdag3S68eBJu-1wJ_K&state=eyJyZXR1cm5UbyI6Ii9ob21lIn0

In [109]:
os.environ['TAVILY_API_KEY'] = getpass.getpass()

··········


In [110]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results = 2)
search_results = search.invoke("Qual a população de Shenzen?")
print(search_results)

[{'url': 'https://worldpopulationreview.com/cities/china/shenzhen', 'content': "Shenzhen's 2024 population is now estimated at 13,311,900. In 1950, the population of Shenzhen was 3,148. Shenzhen has grown by 239,300 in the last year, which represents a 1.83% annual change. These population estimates and projections come from the latest revision of the UN World Urbanization Prospects.These estimates represent the Urban agglomeration of Shenzhen, which typically includes"}, {'url': 'https://pt.wikipedia.org/wiki/Shenzhen', 'content': 'Com uma população de 17,56 milhões em 2020, Shenzhen é a sexta cidade mais populosa da China e é um centro global em tecnologia, pesquisa, indústria, finanças e transporte, sendo que seu porto é o quarto complexo portuário mais movimentado do mundo. A vila de Dafen, no distrito de Longgang, é famosa como uma polo de arte, principalmente pela produção de réplicas de obras-primas e terceirização da criação de arte original.[83][86][87][88] A Orquestra Sinfôni

In [111]:
search_results[0]['content']

"Shenzhen's 2024 population is now estimated at 13,311,900. In 1950, the population of Shenzhen was 3,148. Shenzhen has grown by 239,300 in the last year, which represents a 1.83% annual change. These population estimates and projections come from the latest revision of the UN World Urbanization Prospects.These estimates represent the Urban agglomeration of Shenzhen, which typically includes"

## Usando modelos de chats

In [112]:
from langchain_core.messages import HumanMessage
resp = llm.invoke([HumanMessage(content = "Olá, tudo bem?")])

In [113]:
resp

AIMessage(content='Olá! Tudo bem, e você? Como posso ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 12, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f650476-84b1-4aa3-bcb9-367a4f7a703e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [114]:
tools = [search]

In [115]:
tools

[TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]

Agora podemos ver como é permitir que este modelo faça chamadas com tools. Para permitir isso, usamos .bind_tools para dar ao modelo de linguagem conhecimento dessas ferramentas

In [116]:
models_with_tools = llm.bind_tools(tools)

Agora podemos chamar o modelo. Vamos primeiro chamá-lo com uma mensagem normal e ver como ele responde. Podemos observar tanto o campo content (conteúdo) quanto o campo tool_calls (tools chamadas)

In [117]:
response = models_with_tools.invoke([HumanMessage(content = "Olá")])
response

AIMessage(content='Olá! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 80, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d67b4f6-91e7-4162-8abd-4de634b45cd6-0', usage_metadata={'input_tokens': 80, 'output_tokens': 10, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [118]:
response.content

'Olá! Como posso ajudar você hoje?'

In [119]:
response.tool_calls

[]

Agora, vamos tentar chamá-lo com alguma entrada que esperaria que uma ferramenta fosse chamada.

In [120]:
response = models_with_tools.invoke([HumanMessage(content = "Qual o valor de mercado atual da Blackrock")])

In [121]:
response.content, response.tool_calls

('',
 [{'name': 'tavily_search_results_json',
   'args': {'query': 'valor de mercado atual da Blackrock'},
   'id': 'call_bUbhPQjobWye2h8SSp0RlfXd',
   'type': 'tool_call'}])

Podemos ver que agora não há conteúdo de texto, mas há uma chamada de uma tool

Ele quer que chamemos a ferramenta Tavily Search

Isso ainda não está chamando essa ferramenta - está apenas nos dizendo para fazer isso. Para realmente chamá-lo, precisaremos criar nosso agente.

In [123]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [124]:
#criando agente
agent = create_react_agent(llm = llm, tools = tools, prompt = prompt, stop_sequence = True)
agent_executor = AgentExecutor.from_agent_and_tools(agent = agent, tools = tools, verbose = True, handle_parsing_errors = True)

In [125]:
resp = agent_executor.invoke({"input": "Qual o valor de mercado recente da Blackrock"})



> Entering new AgentExecutor chain...
Para responder a essa pergunta, preciso buscar informações atualizadas sobre o valor de mercado da BlackRock. 
Action: tavily_search_results_json
Action Input: "valor de mercado recente da BlackRock 2023"[{'url': 'https://br.investing.com/pro/BIT:1BLK/explorer/marketcap', 'content': 'Valor de Mercado da BlackRock, Inc. é de 134,7 B. Veja o valor atual e o histórico e compare Valor de Mercado da 1BLK com concorrentes. ... No caso de criptomoedas, mede o preço de negociação mais recente multiplicado pela oferta circulante. ... Valor de mercado; 2023-12-31: NA: NA: 109,4 B: 2024-03-31: NA: NA: 115,1 B: 2024-06-30: NA: NA'}, {'url': 'https://br.advfn.com/jornal/2024/01/resultado-quarto-trimestre-2023-blackrock', 'content': 'A BlackRock (), maior gestora financeira do mundo, surpreendeu o mercado ao divulgar seus resultados do quarto trimestre de 2023, registrando lucro de US$ 9,66 por ação e receita de US$ 4,63'}]O valor de mercado da BlackRock é de 

## LangSmith
Em muitas aplicações construídas com LangChain conterão várias etapas com múltiplas invocações de chamadas de LLM. À medida que essas aplicações se tornam cada vez mais complexas, torna-se crucial poder inspecionar exatamente o que está acontecendo dentro da sua chain ou do agente.

In [126]:
!pip install -q langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.7 MB/s eta 0:00:00


https://smith.langchain.com/o/c74a6757-37ce-472f-b2b1-f20846363120/settings
LANGCHAIN_API_KEY

In [127]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [130]:
from langgraph.prebuilt import create_react_agent as langgraph_react_agent

agent_executor = langgraph_react_agent(llm, tools)
input = "Qual o valor de mercado recente da Blackrock?"

In [131]:
response = agent_executor.invoke({"messages": [HumanMessage(content=input)]})
response['messages']

[HumanMessage(content='Qual o valor de mercado recente da Blackrock?', additional_kwargs={}, response_metadata={}, id='67b5269f-db95-4dda-b953-05c8419179ca'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w4TeZErn88i1UjrbmP8AD7yo', 'function': {'arguments': '{"query":"valor de mercado recente da Blackrock"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 89, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-187232cc-8f7c-495b-9a2f-40cf2fe7abb0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'valor de mercado recente da Bl

In [132]:
response["messages"][-1]

AIMessage(content='O valor de mercado recente da BlackRock é de aproximadamente US$ 148,50 bilhões, conforme informações disponíveis. Outra fonte indica um valor ligeiramente superior, de cerca de US$ 149,06 bilhões. \n\nPara mais detalhes, você pode consultar as seguintes fontes:\n- [ADVFN](https://br.advfn.com/bolsa-de-valores/nyse/BLK/cotacao)\n- [Investing.com](https://br.investing.com/equities/blackrock,-inc.-c)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 349, 'total_tokens': 457, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-f5784be5-2bb0-44e7-80bb-688740cd4376-0', usage_metadata={'input_tokens': 349, 'output_tokens': 108,

In [133]:
response["messages"][-1].content

'O valor de mercado recente da BlackRock é de aproximadamente US$ 148,50 bilhões, conforme informações disponíveis. Outra fonte indica um valor ligeiramente superior, de cerca de US$ 149,06 bilhões. \n\nPara mais detalhes, você pode consultar as seguintes fontes:\n- [ADVFN](https://br.advfn.com/bolsa-de-valores/nyse/BLK/cotacao)\n- [Investing.com](https://br.investing.com/equities/blackrock,-inc.-c)'